In [2]:
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00


In [4]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 56.7 MB/s eta 0:00:00


In [6]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=4fd7c277a3da1613021b23652d3b4a8b8cc311cfde8c4a309b299e2e6f7d2c7c
  Stored in directory: /root/.cache/pip/wheels/81/f2/e0/d578821d723b473d18610ea93810e4a5402463919f07e603d9
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperframe

In [8]:
!pip install python-multipart

In [9]:
from IPython import get_ipython
from IPython.display import display
# %%
from fastapi import FastAPI, File, UploadFile, HTTPException
from pydantic import BaseModel
import joblib, io
import speech_recognition as sr
from googletrans import Translator
from sklearn.exceptions import NotFittedError

# Install necessary libraries if not already installed
# !pip install fastapi uvicorn # This line is usually commented out after installation

app = FastAPI(title="1122 Emergency Classifier API")
app = FastAPI(title="1122 Emergency Classifier API")

# Load models & artifacts
MODEL_PATH    = "/content/best_random_forest_model_new.p"
VECTORIZER_PATH = "/content/tfidf_vectorizer_new.p"
ENCODER_PATH  = "/content/label_encoder_new.p"

model       = joblib.load(MODEL_PATH)
vectorizer  = joblib.load(VECTORIZER_PATH)
label_enc   = joblib.load(ENCODER_PATH)
translator  = Translator()
recognizer  = sr.Recognizer()

class TextRequest(BaseModel):
    text: str

class ClassificationResponse(BaseModel):
    emergency_type: str
    response_message: str

# Your canned responses
RESPONSES = {
    "Fire":    "Fire team dispatched. Evacuate the area immediately.",
    "Medical": "Ambulance is on the way. Please stay calm.",
    "Accident":"Rescue team en route in 5 minutes. Do not move injured.",
    "Normal":  "No emergency detected. Thank you for your call."
}

def classify_text(en_text: str) -> ClassificationResponse:
    tfidf = vectorizer.transform([en_text])
    pred  = model.predict(tfidf)
    label = label_enc.inverse_transform(pred)[0]
    # Map label to response
    resp  = RESPONSES.get(label, "Unable to generate response.")
    return ClassificationResponse(
        emergency_type=label,
        response_message=resp
    )

@app.post("/classify/", response_model=ClassificationResponse)
def classify(req: TextRequest):
    if not req.text.strip():
        raise HTTPException(400, "Text input cannot be empty")
    return classify_text(req.text)

@app.post("/transcribe_and_classify/", response_model=ClassificationResponse)
async def transcribe_and_classify(file: UploadFile = File(...)):
    # only allow wav or mp3
    if file.content_type not in ("audio/wav","audio/x-wav","audio/mpeg"):
        raise HTTPException(400, "Only WAV or MP3 files supported")
    data = await file.read()
    audio = io.BytesIO(data)
    try:
        with sr.AudioFile(audio) as src:
            audio_data = recognizer.record(src)
            urdu_text = recognizer.recognize_google(audio_data, language="ur")
    except sr.UnknownValueError:
        raise HTTPException(422, "Could not understand audio")
    except Exception as e:
        raise HTTPException(500, f"Transcription error: {e}")

    # translate to English
    en_text = translator.translate(urdu_text, src="ur", dest="en").text
    return classify_text(en_text)